# Transformers Pipeline Exploration

**Reference**: [HuggingFace LLM Course — Chapter 1.3](https://huggingface.co/learn/llm-course/chapter1/3)  
**Design doc**: [docs/transformers_pipeline_experiment.md](../docs/transformers_pipeline_experiment.md)

This notebook walks through all 10 `pipeline()` task types taught in the HuggingFace LLM Course,
systematically benchmarking each one and exploring ablation studies.

## The 3-Stage Pipeline Architecture

```
Input (text / image / audio)
        │
        ▼
  Preprocessing   ← Tokenizer / Feature Extractor
        │
        ▼
   Model Forward  ← AutoModel (logits)
        │
        ▼
 Postprocessing   ← Decode, softmax, format
        │
        ▼
Output (human-readable dict)
```

## Task Inventory

| # | Task | Pipeline | Architecture |
|---|------|----------|--------------|
| 1 | Sentiment analysis | `text-classification` | Encoder-only |
| 2 | Zero-shot classification | `zero-shot-classification` | Encoder-decoder |
| 3 | Text generation | `text-generation` | Decoder-only |
| 4 | Mask filling | `fill-mask` | Encoder-only |
| 5 | Named entity recognition | `ner` | Encoder-only |
| 6 | Question answering | `question-answering` | Encoder-only |
| 7 | Summarization | `summarization` | Encoder-decoder |
| 8 | Translation | `translation` | Encoder-decoder |
| 9 | Image classification | `image-classification` | Vision Transformer |
| 10 | Speech recognition | `automatic-speech-recognition` | Encoder-decoder |

> **Note**: First run downloads models (~10 GB total). Subsequent runs use the local cache.

## Setup

In [ ]:
import json
import sys
import time
from pathlib import Path

import torch
from IPython.display import display

# Ensure project root is on the path so `src` is importable
sys.path.insert(0, str(Path.cwd().parent))

# Detect device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Helper: pretty-print results
def show(title, data, indent=2):
    print(f"\n{'─' * 60}")
    print(f"  {title}")
    print(f"{'─' * 60}")
    print(json.dumps(data, indent=indent, default=str, ensure_ascii=False))

# Collect benchmarks for the final comparison table
benchmark_log: list[dict] = []

---
## Experiment 1 — Text Classification (Sentiment Analysis)

**Pipeline**: `text-classification`  
**Model**: `distilbert-base-uncased-finetuned-sst-2-english`  
**Architecture**: Encoder-only (DistilBERT fine-tuned on SST-2)

The model assigns a label (`POSITIVE` / `NEGATIVE`) and a confidence score to each text.

In [ ]:
from transformers import pipeline

# --- Load ---
classifier = pipeline("text-classification", device=DEVICE)

# --- Course examples ---
course_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
course_outputs = classifier(course_inputs)
show("Course examples", dict(zip(course_inputs, course_outputs)))

In [ ]:
# --- Edge cases ---
edge_cases = {
    "neutral": "The weather is somewhat acceptable today.",
    "non_english": "Je suis très heureux d'être ici.",
    "single_word": "Excellent!",
}
for name, text in edge_cases.items():
    result = classifier(text)[0]
    print(f"{name:15s} → {result['label']:8s} ({result['score']:.4f}) | {text!r}")

In [ ]:
# --- Benchmark ---
_ = classifier(course_inputs[0])  # warm-up
times = []
for _ in range(10):
    t0 = time.perf_counter()
    classifier(course_inputs[0])
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "text-classification", "model": "distilbert-sst2",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 10): {bm['warm_latency_ms']} ms")

---
## Experiment 2 — Zero-Shot Classification

**Pipeline**: `zero-shot-classification`  
**Model**: `facebook/bart-large-mnli`  
**Architecture**: Encoder-decoder (BART trained on MNLI)

Classify text into arbitrary labels **without any task-specific fine-tuning**.
Uses Natural Language Inference (NLI): is the text entailed by each candidate label?

### Ablation: Label Count Scaling
How does inference time scale with the number of candidate labels?

In [ ]:
zero_shot = pipeline("zero-shot-classification", device=DEVICE)

# --- Course example ---
sequence = "This is a course about the Transformers library"
labels = ["education", "politics", "business"]
result = zero_shot(sequence, candidate_labels=labels)
show("Course example", {l: round(s, 4) for l, s in zip(result["labels"], result["scores"])})

In [ ]:
# --- Ablation: label count scaling ---
label_pool = ["education", "politics", "business", "technology", "science",
              "sports", "entertainment", "health", "environment", "culture"]

print(f"{'n_labels':>10} | {'latency (ms)':>14} | top label")
print("-" * 45)
for n in [2, 3, 5, 10]:
    sub_labels = label_pool[:n]
    # warm-up
    zero_shot(sequence, candidate_labels=sub_labels)
    t0 = time.perf_counter()
    out = zero_shot(sequence, candidate_labels=sub_labels)
    ms = (time.perf_counter() - t0) * 1000
    print(f"{n:>10} | {ms:>14.1f} | {out['labels'][0]}")

In [ ]:
# Benchmark (3 labels)
_ = zero_shot(sequence, candidate_labels=labels)  # warm-up
times = []
for _ in range(5):
    t0 = time.perf_counter()
    zero_shot(sequence, candidate_labels=labels)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "zero-shot-classification", "model": "bart-large-mnli",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 5, 3 labels): {bm['warm_latency_ms']} ms")

---
## Experiment 3 — Text Generation

**Pipeline**: `text-generation`  
**Default model**: `openai-community/gpt2` (GPT-2)  
**Architecture**: Decoder-only autoregressive LM

### Ablations
1. **Temperature sweep** — {0.7, 1.0, 1.5}: lower = more focused, higher = more creative  
2. **Model comparison** — GPT-2 vs SmolLM2-360M

In [ ]:
generator = pipeline("text-generation", model="openai-community/gpt2", device=DEVICE)

# --- Course example ---
prompt = "In this course, we will teach you how to"
output = generator(prompt, max_new_tokens=50, num_return_sequences=1)
print(f"Prompt : {prompt!r}")
print(f"Output : {output[0]['generated_text']!r}")

In [ ]:
# --- Ablation: temperature sweep ---
print(f"Prompt: {prompt!r}\n")
for temp in [0.7, 1.0, 1.5]:
    out = generator(prompt, max_new_tokens=40, temperature=temp,
                    do_sample=True, num_return_sequences=1)
    text = out[0]["generated_text"][len(prompt):].strip()
    print(f"  temp={temp} → {text!r}")

In [ ]:
# --- Ablation: model comparison ---
smol = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-360M", device=DEVICE)
smol_out = smol(prompt, max_new_tokens=50, num_return_sequences=1)
gpt2_out = generator(prompt, max_new_tokens=50, num_return_sequences=1)

print("GPT-2     :", gpt2_out[0]["generated_text"])
print()
print("SmolLM2   :", smol_out[0]["generated_text"])

In [ ]:
# Benchmark
_ = generator(prompt, max_new_tokens=50)  # warm-up
times = []
for _ in range(5):
    t0 = time.perf_counter()
    generator(prompt, max_new_tokens=50)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "text-generation", "model": "gpt2",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 5, 50 new tokens): {bm['warm_latency_ms']} ms")

---
## Experiment 4 — Fill-Mask

**Pipeline**: `fill-mask`  
**Default model**: `distilroberta-base` (uses `<mask>` token)  
**Architecture**: Encoder-only MLM

### Ablation: mask token / model
- DistilRoBERTa uses `<mask>` 
- BERT-base-cased uses `[MASK]`

In [ ]:
unmasker = pipeline("fill-mask", device=DEVICE)  # default: distilroberta-base

# --- Course example ---
text = "This course will teach you all about <mask> models."
results = unmasker(text)
print(f"Input: {text!r}")
print("\nTop predictions:")
for r in results[:5]:
    print(f"  {r['token_str']:15s} → score={r['score']:.4f} | {r['sequence']!r}")

In [ ]:
# --- Ablation: BERT-base-cased with [MASK] token ---
bert_unmasker = pipeline("fill-mask", model="bert-base-cased", device=DEVICE)
bert_text = "This course will teach you all about [MASK] models."
bert_results = bert_unmasker(bert_text)

print("DistilRoBERTa (<mask>):")
for r in results[:3]:
    print(f"  {r['token_str']:15s} score={r['score']:.4f}")

print("\nBERT-base-cased ([MASK]):")
for r in bert_results[:3]:
    print(f"  {r['token_str']:15s} score={r['score']:.4f}")

In [ ]:
# Benchmark
_ = unmasker(text)  # warm-up
times = []
for _ in range(10):
    t0 = time.perf_counter()
    unmasker(text)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "fill-mask", "model": "distilroberta-base",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 10): {bm['warm_latency_ms']} ms")

---
## Experiment 5 — Named Entity Recognition (NER)

**Pipeline**: `ner`  
**Model**: `dbmdz/bert-large-cased-finetuned-conll03-english`  
**Architecture**: Encoder-only (BERT-large with token classification head)

### Ablation: `grouped_entities`
- `True` → merge subword tokens into whole words/spans  
- `False` → raw per-token predictions

In [ ]:
ner = pipeline("ner", grouped_entities=True, device=DEVICE)

# --- Course example ---
text = "My name is Sylvain and I work at Hugging Face in Brooklyn."
entities = ner(text)
show("Course example (grouped_entities=True)", entities)

In [ ]:
# --- Ablation: grouped vs ungrouped ---
ner_raw = pipeline("ner", grouped_entities=False, device=DEVICE)
raw_entities = ner_raw(text)

print(f"grouped=True  → {len(entities)} entities")
for e in entities:
    print(f"  [{e['entity_group']:3s}] {e['word']!r:20s} ({e['score']:.4f})")

print(f"\ngrouped=False → {len(raw_entities)} tokens")
for e in raw_entities:
    print(f"  [{e['entity']:6s}] {e['word']!r:15s} ({e['score']:.4f})")

In [ ]:
# Benchmark
_ = ner(text)  # warm-up
times = []
for _ in range(10):
    t0 = time.perf_counter()
    ner(text)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "ner", "model": "bert-large-conll03",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 10): {bm['warm_latency_ms']} ms")

---
## Experiment 6 — Question Answering (Extractive QA)

**Pipeline**: `question-answering`  
**Model**: `distilbert-base-cased-distilled-squad`  
**Architecture**: Encoder-only (DistilBERT with span-extraction head)

Extracts the answer span directly from the provided context — no generation involved.

In [ ]:
qa = pipeline("question-answering", device=DEVICE)

# --- Course example ---
result = qa(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)
show("Course example", result)

In [ ]:
# --- Edge cases ---
qa_pairs = [
    {
        "name": "multi-sentence context",
        "question": "What is the capital of France?",
        "context": "France is a country in Western Europe. Its capital city is Paris. The Eiffel Tower is located there.",
    },
    {
        "name": "technical question",
        "question": "What does the pipeline() function do?",
        "context": "The pipeline() function in HuggingFace Transformers is the highest-level entry point for NLP tasks. It encapsulates preprocessing, model forward pass, and postprocessing in a single callable.",
    },
]

for item in qa_pairs:
    res = qa(question=item["question"], context=item["context"])
    print(f"[{item['name']}]")
    print(f"  Q: {item['question']}")
    print(f"  A: {res['answer']!r}  (score={res['score']:.4f}, span=[{res['start']}:{res['end']}])")
    print()

In [ ]:
# Benchmark
bench_input = {"question": "Where do I work?",
               "context": "My name is Sylvain and I work at Hugging Face in Brooklyn"}
_ = qa(**bench_input)  # warm-up
times = []
for _ in range(10):
    t0 = time.perf_counter()
    qa(**bench_input)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "question-answering", "model": "distilbert-squad",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 10): {bm['warm_latency_ms']} ms")

---
## Experiment 7 — Summarization

**Pipeline**: `summarization`  
**Model**: `sshleifer/distilbart-cnn-12-6` (DistilBART-CNN)  
**Architecture**: Encoder-decoder (BART distilled on CNN/DM)

### Ablation: max_length
How does the summary quality change as we allow more output tokens?

In [ ]:
summarizer = pipeline("summarization", device=DEVICE)

# --- Course example ---
article = (
    "America has changed dramatically during recent years. Not only has the number of "
    "graduates in traditional engineering disciplines such as mechanical, civil, "
    "electrical, chemical, and aeronautical engineering declined, but in most of "
    "the premier universities in the United States, many of the valedictorians "
    "and high-ranking students are girls. Increasingly, these students are "
    "particularly in programs such as health and medical related fields, "
    "including premedical programs, medicine, law, business, and computer "
    "sciences. For women, the attractiveness of these majors is that they "
    "offer the opportunity to combine intellectual rigor and career potential "
    "with an altruistic contribution to society. This particular article, "
    "however, is about another major -- the STEM fields, which stands for "
    "science, technology, engineering, and mathematics."
)

result = summarizer(article, max_length=130, min_length=30)
print("Summary:")
print(result[0]["summary_text"])

In [ ]:
# --- Ablation: max_length sweep ---
for max_len in [50, 100, 200]:
    out = summarizer(article, max_length=max_len, min_length=min(20, max_len - 10))
    summary = out[0]["summary_text"]
    print(f"max_length={max_len:3d} → {len(summary.split()):3d} words: {summary[:120]}...")
    print()

In [ ]:
# Benchmark
_ = summarizer(article, max_length=130, min_length=30)  # warm-up
times = []
for _ in range(3):
    t0 = time.perf_counter()
    summarizer(article, max_length=130, min_length=30)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "summarization", "model": "distilbart-cnn-12-6",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 3): {bm['warm_latency_ms']} ms")

---
## Experiment 8 — Translation (French → English)

**Pipeline**: `translation`  
**Model**: `Helsinki-NLP/opus-mt-fr-en` (MarianMT)  
**Architecture**: Encoder-decoder (seq2seq)

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device=DEVICE)

# --- Course example ---
fr_text = "Ce cours est produit par Hugging Face."
result = translator(fr_text)
print(f"FR: {fr_text}")
print(f"EN: {result[0]['translation_text']}")

In [ ]:
# --- Edge cases ---
sentences = [
    "Bonjour, comment allez-vous?",
    "Qu'est-ce que l'intelligence artificielle?",
    "Les modèles de transformateur révolutionnent le traitement du langage naturel.",
]
for fr in sentences:
    en = translator(fr)[0]["translation_text"]
    print(f"FR: {fr}")
    print(f"EN: {en}")
    print()

In [ ]:
# Benchmark
_ = translator(fr_text)  # warm-up
times = []
for _ in range(10):
    t0 = time.perf_counter()
    translator(fr_text)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "translation", "model": "opus-mt-fr-en",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 10): {bm['warm_latency_ms']} ms")

---
## Experiment 9 — Image Classification

**Pipeline**: `image-classification`  
**Model**: `google/vit-base-patch16-224` (ViT-Base)  
**Architecture**: Vision Transformer (patches the image into 16×16 tokens)

The pipeline downloads the image, preprocesses it into 224×224 patches, and
returns top-k class probabilities from ImageNet-21k.

In [ ]:
from IPython.display import Image as IPyImage

img_classifier = pipeline("image-classification", device=DEVICE)

# --- Course example ---
cat_url = (
    "https://huggingface.co/datasets/huggingface/documentation-images"
    "/resolve/main/pipeline-cat-chonk.jpeg"
)
display(IPyImage(url=cat_url, width=300))

results = img_classifier(cat_url)
print("\nTop-5 predictions:")
for r in results[:5]:
    print(f"  {r['label']:40s} {r['score']:.4f}")

In [ ]:
# Benchmark
_ = img_classifier(cat_url)  # warm-up
times = []
for _ in range(5):
    t0 = time.perf_counter()
    img_classifier(cat_url)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "image-classification", "model": "vit-base-patch16-224",
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 5): {bm['warm_latency_ms']} ms")

---
## Experiment 10 — Automatic Speech Recognition (ASR)

**Pipeline**: `automatic-speech-recognition`  
**Model (CPU)**: `openai/whisper-tiny` — ~150 MB, fast on CPU  
**Model (GPU)**: `openai/whisper-large-v3` — ~3 GB, high accuracy  
**Architecture**: Encoder-decoder (Whisper)

### Ablation: CPU model substitution
whisper-tiny (CPU) vs whisper-large-v3 (GPU)

In [ ]:
# Auto-select model based on device
asr_model = "openai/whisper-large-v3" if DEVICE == "cuda" else "openai/whisper-tiny"
print(f"Using ASR model: {asr_model} (device={DEVICE})")

transcriber = pipeline("automatic-speech-recognition", model=asr_model, device=DEVICE)

# --- Course example: MLK speech excerpt ---
audio_url = "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac"
result = transcriber(audio_url)
print(f"\nTranscription: {result['text']!r}")

In [ ]:
# Edge case: shorter audio
short_url = "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac"
short_result = transcriber(short_url)
print(f"Short clip: {short_result['text']!r}")

In [ ]:
# Benchmark
_ = transcriber(audio_url)  # warm-up
times = []
for _ in range(3):
    t0 = time.perf_counter()
    transcriber(audio_url)
    times.append((time.perf_counter() - t0) * 1000)

bm = {"task": "automatic-speech-recognition", "model": asr_model.split("/")[-1],
      "warm_latency_ms": round(sum(times) / len(times), 1)}
benchmark_log.append(bm)
print(f"Warm latency (mean of 3): {bm['warm_latency_ms']} ms")

---
## Results: Benchmark Comparison

Warm inference latency across all 10 tasks (single input, CPU unless GPU detected).

In [ ]:
import pandas as pd

df = pd.DataFrame(benchmark_log)
df = df.sort_values("warm_latency_ms")
df.index = range(1, len(df) + 1)
print(f"Device: {DEVICE}\n")
print(df.to_string(index=True))

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))
colors = ["#4CAF50" if ms < 200 else "#FF9800" if ms < 1000 else "#F44336"
          for ms in df["warm_latency_ms"]]
bars = ax.barh(df["task"], df["warm_latency_ms"], color=colors)
ax.bar_label(bars, fmt="%.0f ms", padding=4)
ax.set_xlabel("Warm latency (ms)")
ax.set_title(f"Pipeline Latency Comparison ({DEVICE})")
ax.invert_yaxis()
plt.tight_layout()
plt.savefig("../results/benchmark_chart.png", dpi=120, bbox_inches="tight")
plt.show()
print("Chart saved to results/benchmark_chart.png")

In [ ]:
# Save benchmark results
import os
os.makedirs("../results", exist_ok=True)
df.to_csv("../results/notebook_benchmarks.csv", index=False)
print("Saved to results/notebook_benchmarks.csv")

---
## Key Takeaways

1. **Encoder-only models** (DistilBERT, DistilRoBERTa) are fastest (~50–150 ms) — ideal for classification and extraction.

2. **Encoder-decoder models** (BART, MarianMT, DistilBART) are slower (~200–2000 ms) — required for generation and seq2seq tasks.

3. **Vision Transformers** (ViT) have comparable latency to text encoders — the image is simply tokenized as patches.

4. **ASR** (Whisper) is the slowest on CPU — audio requires far more computation than text.

5. **Zero-shot classification** scales linearly with label count — each label adds one NLI pass.

6. **Temperature** affects diversity but not speed in text generation — all temperatures have the same forward-pass cost.

7. **The `pipeline()` abstraction** seamlessly handles:
   - Different input types (string, dict, URL, numpy array)
   - Different output schemas per task
   - Device placement (CPU / GPU)
   - Batching and tokenization

---
*Reference*: [HuggingFace LLM Course — Chapter 1.3](https://huggingface.co/learn/llm-course/chapter1/3)